### EECS 730 Project - 1

#### Steps followed
1. Human genome file is huge around 3gb. To extract the sequence by reading this file would be time consuming.
2. Extracted the sequence for a specific chromosome in to temporary fasta file.
3. For the given chromosome, below steps are followed to extract the protein sequence.
4. Extracted sequences for every exon frame, excep for frame = -1. 
5. For Negative strand - 
<br>a. Read the frames, start and end positions in the reverse order.
<br>b. Using exon start and end positions, extract the sequence.
<br>c. Gather sequences for each exon frame in to a list and join in to a single sequence.
<br>d. Compute the starting position of neucleotide sequence using codon end position (cdsEnd from annotation table).
<br>e. Substring the sequence from this new start position.
6. For Positive strand -
<br>a. Read the frames, start and end positions in the normal order.
<br>b. Using exon start and end positions, extract the sequence.
<br>c. Gather sequences for each exon frame in to a list and join in to a single sequence.
<br>d. Compute the starting position of neucleotide sequence using codon start position (cdsSt from annotation table).
<br>e. Substring the sequence from this new start position.    
7. Translate the above extracted squence in to protein sequence using codon table. Stop the translation once a stop codon is read. 
8. Write the protein sequence in to an output file in the below format.

">name:name2 <br>
"Protein sequence

#### Import relevant packages

In [ ]:
# import packages
import os
import Bio
from Bio import SeqIO
from Bio.Seq import Seq
import pandas as pd
import dask.dataframe as dd
from dask.multiprocessing import get

# Print versions
print('The Biopython version is {}..'.format(Bio.__version__))

#### Create the paths for reference files

In [ ]:
# Set the local paths for data
path = r'C:\Users\pmspr\Documents\HS\MS\Sem 4\EECS 730\Data\Project1'
hgfa = os.path.join(path, 'hg38.fa')
annf = os.path.join(path,'Human_genome')
docs = r'C:\Users\pmspr\Documents\HS\MS\Sem 4\EECS 730\Bioinformatics\Project 1\Docs'
test = os.path.join(path,'Testfile.txt')
annfile = os.path.join(path,'annotation.csv')
output = os.path.join(path,'proteinsequence82.txt')

#### Understand the data

In [ ]:
# Get all the chromosomes from the fasta file
chr = []
with open(hgfa) as genome:
    for line in genome:
        if(line[0].strip() == '>'):
            chr.append(line[1:].strip())
print('Total number of chromosomes in database is {}..'.format(len(chr)))

In [ ]:
# Read the annotation file in to a dataframe
hgann = pd.read_csv(annf, delimiter = "\t")

#Save to an excel file for understanding purposes
hgann.to_csv(annfile)

In [ ]:
# Read the genome file and extract sequence for the desired chromosome
count = 0
testfa = os.path.join(path,'testfa_8.fa')
input_seq_iterator = SeqIO.parse(hgfa, "fasta")
short_seq_iterator = (record for record in input_seq_iterator if record.id == 'chr8')
SeqIO.write(short_seq_iterator, testfa, "fasta")

#### Process the sequence data

##### This method complements the given sequence and returns the reversed version of it

In [ ]:
def revcom(s):
    comp_seq = Seq(s)
    comp_seq = comp_seq.complement()
    return comp_seq[::-1]

##### This method extracts the sequecne from the desired chromosome for an exon frame

In [ ]:
# Read the choromose fasta
def getSequence(exon_st, exon_end, strand):
    lc = 0; lst = 1; lend = 60;

    exon_seq = []
    inpst = exon_st + 1 #exon start is zero based
    
    #Read the sequence sequentially
    with open(testfa) as chrom:
        for line in chrom:
            lc += 1
            if (lc == 1): #Ignore the chromosome id
                continue
            # if the start position is found
            if((inpst - lst) > 0 and (inpst - lst) <= 59):
                extract_st = inpst - lst 
                if (exon_end <= lend):
                    exon_seq.append(line[extract_st:exon_end].rstrip("\n"))
                    break
                else:
                    exon_seq.append(line[extract_st:].rstrip("\n"))
                    #print('Line start {} end {}'.format(lst,lend))

            # lines between exon start and end        
            if ((lst > exon_st) and (lend < exon_end)):
                exon_seq.append(line.rstrip("\n"))
                #print('Line start {} end {}'.format(lst,lend))

            # Last line till exon end
            if ((lst > exon_st) and (lend >= exon_end)):
                extract_en = exon_end - lst + 1
                exon_seq.append(line[:extract_en].rstrip("\n"))
                #print('Line start {} end {}'.format(lst,lend))
                break

            lst += 60
            lend += 60

    #join the shrads of sequences in to one        
    sequence = ''.join(exon_seq)
    
    # For positive strand return the normal sequence
    if (strand == '+'):
        return(sequence)
    
    # For negative strand return the complemented & Reversed sequence
    if (strand == '-'):
        return(str(revcom(sequence)))

##### This method prepares indexes to be extracted and call above method for extraction.

In [ ]:
def chromSeq(exstlist, exenlist, exfrlist, strand, cdsSt, cdsEn):
    
    # For negative strands, we read exon data in the reverse order.
    if (strand == '-'):
        
        #Reverses the starts, ends and frames
        exfrlist = exfrlist[::-1]
        exstlist = exstlist[::-1]
        exenlist = exenlist[::-1]
        
        # Call the above method to extract sequence for each exon frame.
        chromSeqlist = [(getSequence(int(j),int(k),strand))for i,j,k in zip(exfrlist,exstlist,exenlist) if (i != '-1')]
        exf      = [(q) for p,q in zip(exfrlist, exenlist) if (p != '-1')]
        
        # Calculate the starting position of neucleotide sequence using the codon end postion for -ve strand.
        cdsVal   = int(exf[0]) - cdsEn
    
    # For positive strands, we read exon data in the normal order. 
    if (strand == '+'):
        
        # call the above method to extract sequecne for each exon frame.
        chromSeqlist = [(getSequence(int(j),int(k),strand))for i,j,k in zip(exfrlist,exstlist,exenlist) if (i != '-1')]
        exf      = [(q) for p,q in zip(exfrlist, exstlist) if (p != '-1')]
        
        # Calculate the starting position of neucleotide sequence using the codon start position for +ve strand
        cdsVal   = cdsSt - int(exf[0])
    
    # Return the gathered sequence list for all the exon frames, for a gene, and its codon start position
    return chromSeqlist, cdsVal

##### This method does following things for a give gene
 1. Prepares the data for a gene and get the genome sequence from above method.
 2. Extracts the neucleotide sequence by using codon start position
 3. Translate the neucleotide sequence in to a protein sequence using codon table.
 4. Writes the protein sequence in to an output file in fasta file format.

In [ ]:
def neucleotideSeq(row, outputfile):
    try:
        
        # Prepare the data for a specific gene
        stlist = row.get('exonStarts').split(",")[:-1]
        enlist = row.get('exonEnds').split(",")[:-1]
        frlist = row.get('exonFrames').split(",")[:-1]
        strand = row.get('strand')
        cdsSt  = row.get('cdsStart')
        cdsEn  = row.get('cdsEnd')
        name   = row.get('name')
        name2  = row.get('name2')
        
        # Call above method to get shards of genome sequence
        chromSeqlist, cdsVal = chromSeq(stlist, enlist, frlist, strand, int(cdsSt), int(cdsEn))
        
        # Join in to a single sequence.
        chromsequence = ''.join(chromSeqlist)
        
        # Using codon start position, get the neucleotide sequence.
        neucleotidesequence = chromsequence[cdsVal:]
        #print(neucleotidesequence)
        
        # Translate neucleotide sequence in to protein sequence.
        nuc_seq = Seq(neucleotidesequence)
        protiensequence = nuc_seq.translate(to_stop=True) # Stop the translation when stop codon is found
        #print(str(protiensequence))
        
        # Write the extracted protein sequence to an output file in fasta file format.
        outputfile.write(str('>' + name + ':' + name2 + '\n'))
        outputfile.write(str(protiensequence + '\n'))

        return (str(protiensequence))
    
    except Exception as ex:
        print("-----------------------------------------------------------------------")
        template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        message = template.format(type(ex).__name__, ex.args)
        print('Error occured for:' + str(name + ':' + name2))

#### Main logic to call above methods for a specific chromosome

In [ ]:
# Extract all the genes, in to a dataframe, for a given chromosome from the annotation file
df = hgann.loc[(hgann['chrom'] == 'chr8') & (hgann['cdsStartStat'] == 'cmpl') & (hgann['cdsEndStat'] == 'cmpl')]

# Covert the pandas dataframe in to a Dask dataframe for multiprocessing.
df_dask = dd.from_pandas(df, npartitions=8)

# Delete the output file if exists
if os.path.exists(output):
  os.remove(output)

# Open the output file in append mode.
outputfile = open(output,"a")

# apply above methods for each row (gene of the given chromosome) to write the protein sequence to output file.
result = df_dask.map_partitions(lambda df: df.apply(lambda row: neucleotideSeq(row, outputfile),axis=1)).compute()

# Close the output file
outputfile.close()

##### Testing

In [ ]:
#testsequence = Seq("MNHSPLKTALAYECFQDQDNSTLALPSDQKMKTGTSGRQRVQEQVMMTVKRQKSKSSQSSTLSHSNRGSMYDGLADNYNYGTTSRSSYYSKFQAGNGSWGYPIYNGTLKREPDNRRFSSYSQMENWSRHYPRGSCNTTGAGSDICFMQKIKASRSEPDLYCDPRGTLRKGTLGSKGQKTTQNRYSFYSTCSGQKAIKKCPVRPPSCASKQDPVYIPPISCNKDLSFGHSRASSKICSEDIECSGLTIPKAVQYLSSQDEKYQAIGAYYIQHTCFQDESAKQQVYQLGGICKLVDLLRSPNQNVQQAAAGALRNLVFRSTTNKLETRRQNGIREAVSLLRRTGNAEIQKQLTGLLWNLSSTDELKEELIADALPVLADRVIIPFSGWCDGNSNMSREVVDPEVFFNATGCLRKRLGMRELLALVPQRATSSRVNLSSADAGRQTMRNYSGLIDSLMAYVQNCVAASRCDDKSVENCMCVLHNLSYRLDAEVPTRYRQLEYNARNAYTEKSSTGCFSNKSDKMMNNNYDCPLPEEETNPKGSGWLYHSDAIRTYLNLMGKSKKDATLEACAGALQNLTASKGLMSSGMSQLIGLKEKGLPQIARLLQSGNSDVVRSGASLLSNMSRHPLLHRVMGNQVFPEVTRLLTSHTGNTSNSEDILSSACYTVRNLMASQPQLAKQYFSSSMLNNIINLCRSSASPKAAEAARLLLSDMWSSKELQGVLRQQGFDRNMLGTLAGANSLRNFTSRF")
testsequence = Seq("MAEKILEKLDVLDKQAEIILARRTKINRLQSEGRKTTMAIPLTFDFQLEFEEALATSASKAISKIKEDKSCSITKSKMHVSFKCEPEPRKSNFEKSNLRPFFIQTNVKNKESESTEPVEEHLKSRSIRPYLYLKDTTEMENAGPLNVLYSQHRQACRRSLGSTDFSPMFNIQSNAHKKEKDSTLFTAQIEKKPRKPLDSVGLLEGDRNKRNKRTQIP")

In [ ]:
if (protiensequence == testsequence):
    print("Same sequence")